# X-tern Data Science Work Sample Assessment

## Information:
When this page first loads, you may have to run all cells. It may take a minute or two to load in the map towards the bottom. 

## Deliverables
1. Clean and organize your data set then present the data as a table or data frame. 
	* Please see the [Data Sets Section](https://colab.research.google.com/drive/1hih0q5XBeb29IIPexmfS6qqjDfOLjdoZ#scrollTo=Y4tcJ4loBlBd) below for all of the data sets cleaned. 
2. Cluster housing, restaurants, event, conference locations to pick out the most convenient coworking space. 
	* Please see the [Map Of Locations Section](https://colab.research.google.com/drive/1hih0q5XBeb29IIPexmfS6qqjDfOLjdoZ#scrollTo=8lRLoLqsBlBk) below.
	* When the map loads, there will be a lot of markers. You can look at the top right corner to be able to add and remove some of these markers.
3. Review the data and draw any conclusions you can find from the data set you gathered. 
4. Present a sample 10-week activities plan with event date, event location and event detail. 
5. Demonstrate your findings using data visualization tool and well written analysis.

## Data Sets

In [ ]:
import pandas
import IPython

address = pandas.read_csv("https://github.com/kcin999/Xtern-Projects/blob/main/address.csv")
IPython.display.display(address)

### Events In Indy
#### Information was found from both Downtown Indy and VisitIndy

In [ ]:
eventsInIndy = pandas.read_csv("https://github.com/kcin999/Xtern-Projects/blob/main/eventsInIndy.csv")
IPython.display.display(eventsInIndy)

#### Information found from EventBrite.
* Idea on how to find this data was found here: [Stevesie Data](https://www.youtube.com/watch?v=GsLuV_Oc9h0). 
* Reason I had to go through this wierd way was due to the fact that EventBrite disabled their /search Endpoint. 
* See file [jsonToCSV](https://github.com/kcin999/Xtern-Projects/blob/main/jsonToCSV.py) in parent directory to see how I cleaned the data from [raw_eventbrite.json](https://github.com/kcin999/Xtern-Projects/blob/main/raw_eventbrite.json) and turned it into [eventbrite.csv](https://github.com/kcin999/Xtern-Projects/blob/main/eventbrite.csv)

In [ ]:
eventBrite = pandas.read_csv("https://github.com/kcin999/Xtern-Projects/blob/main/eventbrite.csv")
IPython.display.display(eventBrite)

### Restaurants

In [ ]:
restaurants = pandas.read_csv("https://github.com/kcin999/Xtern-Projects/blob/main/restaurants.csv")
IPython.display.display(restaurants)

## Map Of Locations
* Markers that are in <span style="color:red">red</span> are co-working areas.
* Markers that are in <span style="color:purple">purple</span> are housing.
* Markers that are in <span style="color:blue">blue</span> are events.
* Markers that are in <span style="color:green">green</span> are restaurants

In [43]:
import requests
import ipyleaflet
import ipywidgets


url = "https://nominatim.openstreetmap.org/search"

addressMarkers = []
addressIcon = ipyleaflet.AwesomeIcon(name="building",marker_color="red")
homeIcon = ipyleaflet.AwesomeIcon(name="home",marker_color="purple")

for index,row in address.iterrows():
	addressParams = {
		'street': row['Address'].split(', ')[0],
		'city:': row['Address'].split(', ')[1],
		'state': row['Address'].split(', ')[2].split(' ')[0],
		'postalcode': row['Address'].split(', ')[2].split(' ')[1],
		'format': 'json'
	}
	#Leverages openstreetmap API to find the latitude and longitude
	response = requests.get(url, params = addressParams)	
	response.raise_for_status()
	results = response.json()[0]
	#Creates the marker with PopUP
	popUpMessage = ipywidgets.HTML()
	popUpMessage.value = results['display_name']
	if (row['Type'] == "Housing"):
		markerToAppend = ipyleaflet.Marker(icon=homeIcon,location=(results['lat'],results['lon']))
	else:
		markerToAppend = ipyleaflet.Marker(icon=addressIcon,location=(results['lat'],results['lon']))
	markerToAppend.popup = popUpMessage
	addressMarkers.append(markerToAppend)

#Markers have to be in a tuble, in order for them to be added to the MarkerCluster
addressMarkers = tuple(addressMarkers)

#Restaurant Markers
restaurantMarkers = []
restaurantIcon = ipyleaflet.AwesomeIcon(name="cutlery",marker_color="green")
for index,row in restaurants.iterrows():
	popUpMessage = ipywidgets.HTML()
	popUpMessage.value = row['restaurant_name']
	markerToAppend = ipyleaflet.Marker(icon=restaurantIcon,location=(row['geo.lat'],row['geo.lon']))
	markerToAppend.popup = popUpMessage
	restaurantMarkers.append(markerToAppend)

#Event Markers
eventIcon = ipyleaflet.AwesomeIcon(name="calendar",marker_color="blue")

eventMarkers = []
for index,row in eventbrite.iterrows():
	popUpMessage = ipywidgets.HTML()
	popUpMessage.value = row['name']
	markerToAppend = ipyleaflet.Marker(icon=eventIcon,location=(row['primary_venue.address.latitude'],row['primary_venue.address.longitude']))
	markerToAppend.popup = popUpMessage
	eventMarkers.append(markerToAppend)

for index,row in eventsInIndy.iterrows():
	addressParams = {
		'street': row['address'].split(', ')[0],
		'city:': row['address'].split(', ')[1],
		'state': row['address'].split(', ')[2].split(' ')[0],
		'postalcode': row['address'].split(', ')[2].split(' ')[1],
		'format': 'json'
	}
	#Leverages openstreetmap API to find the latitude and longitude
	response = requests.get(url, params = addressParams)	
	response.raise_for_status()
	try:
		results = response.json()[0]
		#Creates the marker with PopUP
		popUpMessage = ipywidgets.HTML()
		popUpMessage.value = row['eventName']
		markerToAppend = ipyleaflet.Marker(icon=eventIcon,location=(results['lat'],results['lon']))
		markerToAppend.popup = popUpMessage
		eventMarkers.append(markerToAppend)
	except:
		pass



#centered on the middle of Indianapolis
m = ipyleaflet.Map(center=(39.791000,-86.148003), zoom=10, scroll_wheel_zoom=True)

#Adds the markers
m.add_layer(ipyleaflet.MarkerCluster(name = "Housing and Coworking",markers = addressMarkers))
m.add_layer(ipyleaflet.MarkerCluster(name = "Restaurants",markers = restaurantMarkers))
m.add_layer(ipyleaflet.MarkerCluster(name = "Events",markers = eventMarkers))

m.add_control(ipyleaflet.LayersControl(position="topright"))

#Display the map
IPython.display.display(m)

Map(center=[39.791, -86.148003], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…